In [1]:
import json
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd

In [2]:
data_dir = Path("data")

In [3]:
with open(data_dir / "col_decoder.json", "r") as f:
    col_decoder = json.load(f)
genres = col_decoder['genres'].values()

In [6]:
with open(data_dir / f"genre_names.json", "w") as f:
        json.dump(list(genres), f)

In [82]:
def validate_json(s):
    if len(s) == 2:
        return False
    try:
        json.loads(s.replace("\'", "\""))
        return True
    except:
        return False

In [83]:
data = pd.read_csv(data_dir / "movies_metadata.csv")
data = data[~ (data.genres.isna() | data.vote_average.isna() | data.title.isna() | data.overview.isna() | data.release_date.isna())]
data = data[data.apply(lambda row: validate_json(row.genres), axis=1)]
dtind = pd.DatetimeIndex(data.release_date)
data['year'] = dtind.year
data['ym'] = dtind.strftime("%Y-%m")
data = data[(data.year >= 1980) & (data.year <= 2016)]
data.sort_values("ym", inplace=True)
print(len(data))
data.head(1)

c:\Users\Владислав\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


30783


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,ym
4698,False,"{'id': 41649, 'name': 'The Party Collection', ...",0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,166,tt0082100,fr,La Boum,Le Boum is the story of a 13-year-old French g...,...,110.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,The Party,False,6.4,225.0,1980,1980-01


Ratings by genre

In [11]:
for genre in tqdm(genres):
    res = data[data.apply(lambda row: any([genre_row["name"] == genre for genre_row in json.loads(row.genres.replace("\'", "\""))]), axis=1)]\
            .groupby("ym", as_index=False).agg(rate=("vote_average", "mean"))
    res["ym"] = pd.DatetimeIndex(res['ym']).strftime("%Y-%m-%d")

    with open(data_dir / f"genres_{genre}_rates.json", "w") as f:
        json.dump(res.to_json(orient="records"), f)

100%|██████████| 20/20 [00:10<00:00,  1.86it/s]


Words stats by genre

In [84]:
data["text"] = data["title"] + " " + data["overview"]

In [86]:
import string
from nltk.corpus import stopwords
from collections import Counter
import spacy


sp = spacy.load('en_core_web_sm')
signs = string.punctuation
for i in range(10):
    signs += str(i)
table = str.maketrans('', '', signs)

stop_words = set(stopwords.words('english'))
stop_words |= {'-PRON-'}

def preprocess_text(text):
    words = text.split()
    words = [word.lower() for word in words]
    # punctuation and digits
    words = [w.translate(table) for w in words if 
                   len(w.translate(table)) > 1]
    # lematization
    words = [token.lemma_ for token in sp(' '.join(words))]
    # stop words
    words = [w for w in words if not w in stop_words]
    return list(sorted(Counter(words).items(), reverse=True, key=lambda x: x[1]))[:20]

In [87]:
for genre in tqdm(genres):
    res = data[data.apply(lambda row: any([genre_row["name"] == genre for genre_row in json.loads(row.genres.replace("\'", "\""))]), axis=1)]\
            .groupby("year", as_index=False).agg(text=("text", lambda x: " ".join(x)))
    res["counts"] = res["text"].map(preprocess_text)
    res.drop(columns=["text"], inplace=True)

    with open(data_dir / f"genres_{genre}_wc.json", "w") as f:
        json.dump(res.to_json(orient="records"), f)

100%|██████████| 20/20 [10:09<00:00, 30.49s/it]
